## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Krasnozatonskiy,RU,61.6758,50.9794,8.82,92,20,6.71,few clouds
1,1,Carutapera,BR,-1.1950,-46.0200,75.11,93,100,4.00,overcast clouds
2,2,Valparaiso,CL,-33.0393,-71.6273,66.47,72,0,5.75,clear sky
3,3,Ciudad Bolivar,VE,8.1222,-63.5497,74.62,78,66,9.60,broken clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,44.26,87,0,5.75,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Carutapera,BR,-1.1950,-46.0200,75.11,93,100,4.00,overcast clouds
3,3,Ciudad Bolivar,VE,8.1222,-63.5497,74.62,78,66,9.60,broken clouds
6,6,Tamandare,BR,-8.7597,-35.1047,77.58,74,36,6.06,scattered clouds
10,10,Rikitea,PF,-23.1203,-134.9692,78.49,73,13,7.85,few clouds
16,16,Veraval,IN,20.9000,70.3667,73.63,45,9,5.75,clear sky
18,18,Acu,BR,-5.5767,-36.9086,75.85,80,54,5.17,broken clouds
22,22,Gunnedah,AU,-30.9833,150.2500,74.82,37,30,19.71,scattered clouds
25,25,Arraial Do Cabo,BR,-22.9661,-42.0278,75.47,89,86,20.38,overcast clouds
26,26,Mbandaka,CD,0.0487,18.2603,78.12,55,100,3.24,overcast clouds
38,38,Cape Town,ZA,-33.9258,18.4232,73.11,59,0,4.61,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                165
City                   165
Country                165
Lat                    165
Lng                    165
Max Temp               165
Humidity               165
Cloudiness             165
Wind Speed             165
Current Description    165
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Carutapera,BR,-1.1950,-46.0200,75.11,93,100,4.00,overcast clouds
3,3,Ciudad Bolivar,VE,8.1222,-63.5497,74.62,78,66,9.60,broken clouds
6,6,Tamandare,BR,-8.7597,-35.1047,77.58,74,36,6.06,scattered clouds
10,10,Rikitea,PF,-23.1203,-134.9692,78.49,73,13,7.85,few clouds
16,16,Veraval,IN,20.9000,70.3667,73.63,45,9,5.75,clear sky
...,...,...,...,...,...,...,...,...,...,...
681,681,Caravelas,BR,-17.7125,-39.2481,75.78,83,100,5.55,overcast clouds
687,687,Presidencia Roque Saenz Pena,AR,-26.7852,-60.4388,70.38,95,99,3.67,overcast clouds
688,688,Mfou,CM,3.7159,11.6427,70.20,68,87,1.63,overcast clouds
689,689,Santa Maria Da Boa Vista,BR,-8.8078,-39.8256,73.99,73,22,11.95,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Carutapera,BR,75.11,overcast clouds,-1.1950,-46.0200,
3,Ciudad Bolivar,VE,74.62,broken clouds,8.1222,-63.5497,
6,Tamandare,BR,77.58,scattered clouds,-8.7597,-35.1047,
10,Rikitea,PF,78.49,few clouds,-23.1203,-134.9692,
16,Veraval,IN,73.63,clear sky,20.9000,70.3667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Carutapera,BR,75.11,overcast clouds,-1.1950,-46.0200,Lidera
3,Ciudad Bolivar,VE,74.62,broken clouds,8.1222,-63.5497,Hotel Edi
6,Tamandare,BR,77.58,scattered clouds,-8.7597,-35.1047,Pousada 100 Passos
10,Rikitea,PF,78.49,few clouds,-23.1203,-134.9692,People ThankYou
16,Veraval,IN,73.63,clear sky,20.9000,70.3667,Lords Inn Somnath
...,...,...,...,...,...,...,...
681,Caravelas,BR,75.78,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
687,Presidencia Roque Saenz Pena,AR,70.38,overcast clouds,-26.7852,-60.4388,Hotel Gualok
688,Mfou,CM,70.20,overcast clouds,3.7159,11.6427,Maison Du Juge Noah
689,Santa Maria Da Boa Vista,BR,73.99,few clouds,-8.8078,-39.8256,Fênix Hotel e Restaurante


In [17]:
hotel_df= hotel_df.replace('', np.nan)

In [25]:
hotel_df.dropna(how = "any", inplace=True)

In [26]:
hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
645,Anloga,GH,79.38,light rain,5.7947,0.8973,Pin Drop Hotel
653,Nong Khai,TH,79.38,overcast clouds,17.9167,103.0833,จารุวรรณ วงชัยยา
662,Mabaruma,GY,73.22,broken clouds,8.2000,-59.7833,Monty Brooms Guest House
666,Novita,CO,72.32,light rain,4.9551,-76.6053,B/ Buenos Aires Sector Las Curvitas
679,Ewo,CG,76.84,overcast clouds,-0.8725,14.8206,Residence Elonda Ewo
681,Caravelas,BR,75.78,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
687,Presidencia Roque Saenz Pena,AR,70.38,overcast clouds,-26.7852,-60.4388,Hotel Gualok
688,Mfou,CM,70.20,overcast clouds,3.7159,11.6427,Maison Du Juge Noah
689,Santa Maria Da Boa Vista,BR,73.99,few clouds,-8.8078,-39.8256,Fênix Hotel e Restaurante
690,Sao Joao De Pirabas,BR,75.47,overcast clouds,-0.7747,-47.1772,POUSADA SOL E MAR


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))